### Python module/package imports for this chapter

In [1]:
import sys, math, collections, itertools, multiprocessing, gzip

In [2]:
import numpy as np

import matplotlib
import matplotlib.pyplot as pp

%matplotlib inline

In [3]:
%load_ext line_profiler
%load_ext memory_profiler

## Computing Mandelbrot in parallel 

In [4]:
def compute_mandel_numpy(c,maxit=256):
    escaped = np.full_like(c,np.inf,'d')
    z = np.zeros_like(c,'c16')
    
    for it in range(1,maxit):
        z = np.where(escaped == np.inf,z**2 + c,0)        
        escaped[np.abs(z) > 2.0] = it

    return escaped
        
def run_mandel_numpy(extent=[-2.0,1.0,-1.25,1.25],res=256,maxit=256):
    xs = np.linspace(extent[0],extent[1],res)
    ys = np.linspace(extent[2],extent[3],res)
    
    c = xs[:,np.newaxis] + 1j*ys[np.newaxis,:]

    escaped = compute_mandel_numpy(c,maxit)
    
    return escaped / maxit

In [5]:
def plot_mandel(extent=[-2.0,1.0,-1.25,1.25],res=256,maxit=256,run_mandel=run_mandel_numpy):
    pp.figure(figsize=(6,6))

    cmap = matplotlib.cm.coolwarm
    cmap.set_bad('k')

    pp.imshow(run_mandel(extent,res,maxit).T,extent=extent,
              cmap=cmap,norm=matplotlib.colors.PowerNorm(0.6),
              interpolation='none',origin='lower')

In [6]:
plot_mandel(run_mandel=run_mandel_numpy)

In [7]:
%timeit run_mandel_numpy()

10 loops, best of 3: 159 ms per loop


In [8]:
def run_mandel_rows(extent=[-2.0,1.0,-1.25,1.25],res=256,maxit=256):
    xs = np.linspace(extent[0],extent[1],res)
    ys = np.linspace(extent[2],extent[3],res)
    
    c = xs[:,np.newaxis] + 1j*ys[np.newaxis,:]

    escaped_rows = [compute_mandel_numpy(c_row,maxit) for c_row in c]
    
    return np.vstack(escaped_rows) / maxit

In [9]:
%timeit run_mandel_rows()

1 loop, best of 3: 844 ms per loop


In [10]:
def run_mandel_chunked(extent=[-2.0,1.0,-1.25,1.25],res=256,maxit=256):
    xs = np.linspace(extent[0],extent[1],res)
    ys = np.linspace(extent[2],extent[3],res)
    
    c = xs[:,np.newaxis] + 1j*ys[np.newaxis,:]

    chunksize = 32
    escaped_chunks = [compute_mandel_numpy(c[i:i+chunksize,:],maxit) for i in range(0,res,chunksize)]
    
    return np.vstack(escaped_chunks) / maxit

In [11]:
%timeit run_mandel_chunked()

1 loop, best of 3: 199 ms per loop


In [14]:
def run_mandel_mp(extent=[-2.0,1.0,-1.25,1.25],res=256,maxit=256):
    xs = np.linspace(extent[0],extent[1],res)
    ys = np.linspace(extent[2],extent[3],res)
    
    c = xs[:,np.newaxis] + 1j*ys[np.newaxis,:]

    chunksize = 32
    
    pool = multiprocessing.Pool(processes=4)
    escaped_chunks = pool.map(compute_mandel_numpy,
                              (c[i:i+chunksize,:] for i in range(0,res,chunksize)))
    pool.close()
    
    return np.vstack(escaped_chunks) / maxit

In [15]:
plot_mandel(run_mandel=run_mandel_mp)

In [16]:
%timeit run_mandel_mp()

10 loops, best of 3: 70 ms per loop


In [17]:
%timeit run_mandel_chunked(res=1024)

1 loop, best of 3: 2.65 s per loop


In [18]:
%timeit run_mandel_mp(res=1024)

1 loop, best of 3: 744 ms per loop
